In [1]:
import uproot
import encode

In [2]:
fName='root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/NewNtuples/ntuple_hgcalNtuples_vbf_hmm_200PU_1.root'
_tree = uproot.open(fName,xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))['hgcalTriggerNtuplizer/HGCalTriggerNtuple']

In [3]:
df = _tree.pandas.df( ['tc_subdet','tc_zside','tc_layer','tc_wafer','tc_cell','tc_data','tc_uncompressedCharge','tc_compressedCharge'],entrystart=0,entrystop=10)
df.columns = ['subdet','zside','layer','wafer','triggercell','data','uncompressedCharge','compressedCharge']

In [4]:
df.set_index(['subdet','zside','layer','wafer','triggercell'],append=True,inplace=True)
df.reset_index('subentry',drop=True,inplace=True)
df.sort_index(inplace=True)

In [5]:
df.head()

data  uncompressedCharge  \
entry subdet zside layer wafer triggercell                             
0     3      -1    1     7     0             608                 595   
                               1             100                 101   
                               2             608                 592   
                               3              64                  63   
                               4             184                 181   

                                            compressedCharge  
entry subdet zside layer wafer triggercell                    
0     3      -1    1     7     0                          99  
                               1                          57  
                               2                          99  
                               3                          48  
                               4                          71

In [6]:
df[df.compressedCharge==df.compressedCharge.max()]

,,,,,,data,uncompressedCharge,compressedCharge
entry,subdet,zside,layer,wafer,triggercell,,,
2,3,1,7,42,12,38912,38162,195


In [7]:
print('Uncompressed branch, binary & length')
raw = bin(38162)[2:]
print(raw)
print(len(raw))
print()
print('Data branch, binary')
print(bin(38912)[2:])
print()
print('Compressed branch, binary')
print(bin(195)[2:])
print()
print("With Jim's encoding")
print(encode.encode(44105,True,1,4,3,True))

Uncompressed branch, binary & length
1001010100010010
16

Data branch, binary
1001100000000000

Compressed branch, binary
11000011

With Jim's encoding
1100011


In [8]:
df['encodedCharge'] = df.uncompressedCharge.apply(encode.encode,args=(True,1,4,3,True))

In [9]:
df.head()

data  uncompressedCharge  \
entry subdet zside layer wafer triggercell                             
0     3      -1    1     7     0             608                 595   
                               1             100                 101   
                               2             608                 592   
                               3              64                  63   
                               4             184                 181   

                                            compressedCharge encodedCharge  
entry subdet zside layer wafer triggercell                                  
0     3      -1    1     7     0                          99       0110001  
                               1                          57       0011101  
                               2                          99       0110001  
                               3                          48       0011000  
                               4                          71       0100011

In [10]:
def decode(val):
    exp = val[:4]
    mant = val[4:]
    shift = int(exp,2)
    data = (1<<(shift+3)) + (int(mant,2)<<(shift))
    return data

In [11]:
#get geometry information

geomName = "root://cmseos.fnal.gov//store/user/dnoonan/HGCAL_Concentrator/triggerGeomV9.root"
geomTree = uproot.open(geomName,xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))["hgcaltriggergeomtester/TreeTriggerCells"]


In [12]:
geomDF = geomTree.pandas.df(['subdet','zside','layer','wafer','triggercell','c_n'])


In [13]:
geomDF.set_index(['subdet','zside','layer','wafer','triggercell'],inplace=True)
geomDF['isHDM'] = geomDF.c_n>4
geomDF.sort_index(inplace=True)

In [14]:
#### Get single wafer (event 0, subdet 3, zside 1, layer 5, wafer 99)
dfWafer = df.loc[0,3,1,5,99]
geomDFWafer = geomDF.loc[3,1,5,99]

In [15]:
dfWafer

,data,uncompressedCharge,compressedCharge,encodedCharge
triggercell,,,,
4,76,74,51,0011001
7,20,20,20,0001010
9,27,27,27,0001110
16,28,28,28,0001110
17,288,282,82,0101001
20,192,194,72,0100100
21,288,293,82,0101001
22,54,53,43,0010110
24,464,461,93,0101110


In [16]:
geomDFWafer

,c_n,isHDM
triggercell,,
0,4,False
1,4,False
2,4,False
3,4,False
4,4,False
5,4,False
6,4,False
7,4,False
8,4,False


In [17]:
geomDFWafer['encodedCharge'] = dfWafer['encodedCharge']

/cvmfs/sft.cern.ch/lcg/views/LCG_95apython3/x86_64-centos7-gcc8-opt/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
geomDFWafer

,c_n,isHDM,encodedCharge
triggercell,,,
0,4,False,NaN
1,4,False,NaN
2,4,False,NaN
3,4,False,NaN
4,4,False,0011001
5,4,False,NaN
6,4,False,NaN
7,4,False,0001010
8,4,False,NaN


In [19]:
geomDFWafer.fillna('0000000',inplace=True)

/cvmfs/sft.cern.ch/lcg/views/LCG_95apython3/x86_64-centos7-gcc8-opt/lib/python3.6/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [20]:
geomDFWafer

,c_n,isHDM,encodedCharge
triggercell,,,
0,4,False,0000000
1,4,False,0000000
2,4,False,0000000
3,4,False,0000000
4,4,False,0011001
5,4,False,0000000
6,4,False,0000000
7,4,False,0001010
8,4,False,0000000


In [21]:
##### Next steps:

### Assign a mapping to links (add column to dfGeom for link and column for link_channel)

### Build stings

### Repeat on different wafers events